In [34]:
# This is the script to populate driver's Data
# {driver_id, time, curr_lat, curr_long, dest, load}

import csv
import random
from kafka import KeyedProducer, KafkaClient
from datetime import datetime
import time
import json
import random 
from elasticsearch import Elasticsearch

boundaries_file = "boundaries.csv"
last_uid = 0 
kafka = KafkaClient('localhost:9092')
producer = KeyedProducer(kafka)
city = 'NYC'

es = Elasticsearch(['ip-172-31-0-107', 'ip-172-31-0-100', \
                    ' ip-172-31-0-105', 'ip-172-31-0-106'], \
                   port=9200)



def loadBoundaries(boundaries_file):
    boundaries = {}
    with open(boundaries_file, 'r') as f:
        reader = csv.reader(f, delimiter = ',')
        for row in reader:
            city = row[0]
            fence = row[1:] 
            boundaries.update({city:fence})
    f.close()
    return boundaries

def simulateTrip(id, c):
    step = 4
    q = es.get(index='driver', doc_type='rolling', id=id, ignore=[404, 400])
    if q['found'] and (q['_source']['status'] in ['ontrip', 'pickup']): 
        d = q['_source']['destination']
        c = q['_source']['location']
        return [c[0] + ((c[0] - d[0])/step), c[1] + ((c[1] - d[1])/step)]
    else:
        return [random.uniform(float(bnd[0]), float(bnd[2])), \
                random.uniform(float(bnd[1]),float(bnd[3]))]
        
        
def generateDriver(city):
    global last_uid 
    bnd = bound[city]
    last_uid += 1
    curr_lat = random.uniform(float(bnd[0]), float(bnd[2]))
    curr_long = random.uniform(float(bnd[1]),float(bnd[3]))
    
    driver_mapping ={ 
            'name': 'driver_{}'.format(last_uid),
            'id': random.randint(1, 20),
            'status': 'idle',
            'location': simulateTrip(),
            'ctime': str(datetime.now()),
            'p1': None,
            'p2': None,
            'destination': None,
            'destinationid': None,
            'altdest1': None,
            'altdest1id': None,
            'altdest2': None,
            'altdest2id': None,
        }

    return(driver_mapping)

bound = loadBoundaries(boundaries_file)
for n in range(20):
    driver = generateDriver(city)
    u_json = json.dumps(driver).encode('utf-8')
    key = json.dumps(city).encode('utf-8')
    print('{}'.format(driver))
    producer.send(b'driver', key, u_json) 
    time.sleep(2)


IOError: [Errno 2] No such file or directory: 'boundaries.csv'

In [35]:

import csv
import random
from kafka import KeyedProducer, KafkaClient
from datetime import datetime
import time
import json
import random 
from elasticsearch import Elasticsearch

boundaries_file = "boundaries.csv"
last_uid = 0 
kafka = KafkaClient('localhost:9092')
producer = KeyedProducer(kafka)
city = 'NYC'

es = Elasticsearch(['ip-172-31-0-107', 'ip-172-31-0-100', ' ip-172-31-0-105', 'ip-172-31-0-106'], port=9200)



In [37]:
    q = es.get(index='driver', doc_type='rolling', id=5, ignore=[404, 400])


In [38]:
if q['found'] and (q['_source']['status'] in ['ontrip', 'pickup']): 
        d = q['_source']['destination']
        c = q['_source']['location']
        print [c[0] + ((c[0] - d[0])/step_to_dest), d[1] + ((d[1] - d[1])/step_to_dest)]

In [16]:

#from pyspark import SparkContext
#from pyspark import SparkConf
#from pyspark.streaming import StreamingContext
#from pyspark.streaming.kafka import KafkaUtils

from kafka import KafkaConsumer
from elasticsearch import Elasticsearch
import os
import json
from datetime import datetime


#sc = SparkContext(appName="trip")
#ssc = StreamingContext(sc, 4)
es = Elasticsearch(['ip-172-31-0-107', 'ip-172-31-0-100', ' ip-172-31-0-105', 'ip-172-31-0-106'], port=9200)

class driver(object):
    def __init__(self, *arg, **kwargs):
        for item in arg:
            for key in item:
                setattr(self, key, item[key])
        for key in kwargs:
            setattr(self, key, kwargs[key])
        try:
            res = datetime.strptime("{}".format(unicode(self.ctime)), '%Y-%m-%d %H:%M:%S.%f')
            self.ctime = res.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
        except:
            pass
    def jsonFormat(self):
        return(json.dumps(self.__dict__))
    def isKnown(self):
        res = es.get(index='driver', doc_type='rolling', id=self.id, ignore=[404, 400])
        return(res['found'])
    def store(self):
        res = es.create(index='driver', doc_type='rolling', id=self.id, body=self.jsonFormat())
        return(res['created'])
    def update(self):
        q = '{{"doc": {}}}'.format(self.jsonFormat())
        res = es.update(index='driver', doc_type='rolling', id=self.id, body=q)
        return(res['_version'])
    def nearbyPassengers(self):
        geo_query = geoQuery(self.location, "5km")
        res = es.search(index='passenger', doc_type='rolling', body=geo_query )
        nearby = []
        for i in (res['hits']['hits']):
            nearby.append(i['_id'])
        return(nearby)
    
    def assignPassenger(self):
        if len(self.nearbyPassengers())>0: 
            p = getPassenger(self.nearbyPassengers()[0]) 
            p.status = 'pickup'
            self.status = 'pickup'
            p.driver = self.id
            self.destination = p.location
            self.destinationid = p.id
            p.update()
            self.update()
            return True
        else:
            return False

    def loadPassenger(self, p):
        if self.p1 == None: 
            self.p1 = p.id
            p.status = 'ontrip'
            self.status = 'ontrip'
            
        elif self.p2 == None:
            self.p2 = p.id
            p1 = getPassenger(self.p1)
            p1.match = p.id
            p.match = p1.id
            p1.status = 'match'
            p.status = 'match'
            self.status = 'full'
            p1.update()
        else:
            print('Cab is full')
            return False
        p.driver = self.id
        self.destination = p.destination
        self.destinationid = p.destinationid
        p.update()
        self.update()
        return True

    

class passenger(object):
    def __init__(self, *arg, **kwargs):
        for item in arg:
            for key in item:
                setattr(self, key, item[key])
        for key in kwargs:
            setattr(self, key, kwargs[key])
        try:
            res = datetime.strptime("{}".format(unicode(self.ctime)), '%Y-%m-%d %H:%M:%S.%f')
            self.ctime = res.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
        except:
            pass
    def jsonFormat(self):
        return(json.dumps(self.__dict__))
    def isKnown(self):
        res = es.get(index='passenger', doc_type='rolling', id=self.id, ignore=[404, 400])
        return(res['found'])
    def store(self):
        res = es.create(index='passenger', doc_type='rolling', id=self.id, body=self.jsonFormat())
        return(res['created'])
    def update(self):
        q = '{{"doc": {}}}'.format(self.jsonFormat())        
        res = es.update(index='passenger', doc_type='rolling', id=self.id, body=q)
        return(res['_version'])

def geoQuery(location, distance):
    geo_query = { "from" : 0, "size" : 3,
                 "_source":{"include": [ "_id" ]},
                 "query": {
            "filtered": {
                "query" : {
                    "term" : {"status": "wait"}
                },
                "filter": {
                    "geo_distance": {
                        "distance":      distance,
                        "distance_type": "plane", 
                        "location": location }
                           }
                         }
                           }
                }

    return(geo_query)

def getPassenger(p_id):
    res = es.get(index='passenger', doc_type='rolling', id=p_id, ignore=404)
    return(passenger(res['_source'])) if res['found'] else res['found']

def getDriver(p_id):
    res = es.get(index='driver', doc_type='rolling', id=p_id, ignore=404)
    return(driver(res['_source'])) if res['found'] else res['found']
    
def sanityCheck(driver):
    if driver.isKnown():
        driverRecord = getDriverRecord(driver.id)
        return(driver.time > driverRecord.time)
    else:
        return(True)    

def updateLocation(self):
    if isinstance(self, 'driver'):
        obj_ = getDriver(self.id)
    elif isinstance(self, 'passenger'):
        obj_ = getPassenger(self.id)    
    else:
        return(False)
    obj_.location = self.location
    obj_.update()
    return(True)

def arrived(d):
    p = getPassenger(d.p1)
    d.p1 == None
    d.p2 == None
    d.destination == None
    d.destinationid == None
    d.status == 'idle'
    p.status = 'arrived'
    self.update()
    p.update()
    if d.p2: 
        p2 = getPassenger(d.p2)
        p2.status == 'arrived'
        p2.update()
    return(True)


'''
    Driver:
    1. Check if timestamp make sense
    2. Check if driver exists in DB, if doesn't exists, create
    3. If cab's capacity is not zero, look for nearby requests
        1. If passenger request matched:
            1. Change status from idle to on trip (if necessary)
            2. Re-route / Set destination to passenger
            3. Save information to ElasticSearch
        2. If no nearby passenger:
            1. Update current location
    4. If cab's capacity is zero:
        Check if current location matches with destination:
        1. If matched, empty the cab and mark to idle
            1. Update number of trip
            2. Send trip info to kafka (for archive)
        2. If not matched, update current location
'''
def pipe(x):
    d = driver(x)
    if d.isKnown():
        if d.status in ['idle']:
            d.assignPassenger()
        elif d.status in ['pickup']:
            if not d.location == d.destination:
                d.update()
            else:
                p = getPassenger(d.p2) if d.p2 else getPassenger(d.p1)
                d.loadPassenger(d, p)

        elif d.status in ['ontrip']:
            if d.location == d.destination:
                arrived(d)
            elif not d.p2: 
                p = getPassenger(d.p1)
                p.location = d.location
                p.update()
                d.assignPassenger()
            else:
                p = getPassenger(d.p1)
                p.location = d.location
                p2 = getPassenger(d.p2)
                p2.location = d.location
                p.update()
                p2.update()
    else:
        d.store()



In [5]:
d = getDriver(5)
d.nearbyPassengers()

[u'25', u'29', u'24']

In [22]:
# This is the script to populate driver's Data
# {driver_id, time, curr_lat, curr_long, dest, load}

import csv
import random
from kafka import KeyedProducer, KafkaClient
from datetime import datetime
import time
import json
import random 
from elasticsearch import Elasticsearch

boundaries_file = "../kafka/boundaries.csv"
last_uid = 0 
kafka = KafkaClient('localhost:9092')
producer = KeyedProducer(kafka)
city = 'NYC'
total_drivers = 20
sleep = 2
step_to_dest = 4

es = Elasticsearch(['ip-172-31-0-107', 'ip-172-31-0-100', \
                    ' ip-172-31-0-105', 'ip-172-31-0-106'], \
                   port=9200)



def loadBoundaries(boundaries_file):
    boundaries = {}
    with open(boundaries_file, 'r') as f:
        reader = csv.reader(f, delimiter = ',')
        for row in reader:
            city = row[0]
            fence = row[1:] 
            boundaries.update({city:fence})
    f.close()
    return boundaries

def simulateTrip(id, city):
    step = 4
    bnd = bound[city]
    q = es.get(index='driver', doc_type='rolling', id=id, ignore=[404, 400])
    if q['found'] and (q['_source']['status'] in ['ontrip', 'pickup']): 
        d = q['_source']['destination']
        c = q['_source']['location']
        return [c[0] + ((c[0] - d[0])/step_to_dest), d[1] + ((d[1] - d[1])/step_to_dest)]
    else:
        return [random.uniform(float(bnd[0]), float(bnd[2])), \
                random.uniform(float(bnd[1]),float(bnd[3]))]
        
        
def generateDriver(city):
    global last_uid 
    last_uid += 1
    d_id = random.randint(1, total_drivers)
    d_id = 6
    
    driver_mapping ={ 
            'name': 'driver_{}'.format(d_id),
            'id': d_id,
            'status': 'idle',
            'location': simulateTrip(d_id, city),
            'ctime': str(datetime.now()),
            'p1': None,
            'p2': None,
            'destination': None,
            'destinationid': None,
            'altdest1': None,
            'altdest1id': None,
            'altdest2': None,
            'altdest2id': None,
        }

    q = es.get(index='driver', doc_type='rolling', id=d_id, ignore=[404, 400])
    if q['found'] and (q['_source']['status'] in ['ontrip', 'pickup']): 
        driver_mapping['status'] = q['_source']['status']
        driver_mapping['p1'] = q['_source']['p1']
        driver_mapping['p2'] = q['_source']['p2']
        driver_mapping['destination'] = q['_source']['destination']
    return(driver_mapping)



In [14]:
bound = loadBoundaries(boundaries_file)
for n in range(total_drivers):
    driver = generateDriver(city)
    u_json = json.dumps(driver).encode('utf-8')
    key = json.dumps(city).encode('utf-8')
    print('{}'.format(driver))
    producer.send(b'driver', key, u_json) 
    #time.sleep(sleep)


{'status': 'idle', 'altdest2id': None, 'altdest2': None, 'altdest1': None, 'id': 19, 'name': 'driver_19', 'p2': None, 'p1': None, 'ctime': '2016-09-23 18:40:41.999492', 'altdest1id': None, 'destination': None, 'destinationid': None, 'location': [40.816152881233066, -73.99157309722851]}
{'status': 'idle', 'altdest2id': None, 'altdest2': None, 'altdest1': None, 'id': 10, 'name': 'driver_10', 'p2': None, 'p1': None, 'ctime': '2016-09-23 18:40:42.092792', 'altdest1id': None, 'destination': None, 'destinationid': None, 'location': [40.73355967958459, -73.96829113433932]}
{'status': 'idle', 'altdest2id': None, 'altdest2': None, 'altdest1': None, 'id': 7, 'name': 'driver_7', 'p2': None, 'p1': None, 'ctime': '2016-09-23 18:40:42.098255', 'altdest1id': None, 'destination': None, 'destinationid': None, 'location': [40.79516697383348, -73.93150323963204]}
{'status': 'idle', 'altdest2id': None, 'altdest2': None, 'altdest1': None, 'id': 10, 'name': 'driver_10', 'p2': None, 'p1': None, 'ctime': '201

In [36]:
d = getDriver(6)

In [19]:
d.nearbyPassengers()

[u'14', u'29', u'19']

In [31]:
e = generateDriver('NYC')

In [24]:


    q = es.get(index='driver', doc_type='rolling', id=d_id, ignore=[404, 400])
    if q['found'] and (q['_source']['status'] in ['ontrip', 'pickup']): 
        driver_mapping['status'] = q['_source']['status']
        driver_mapping['p1'] = q['_source']['p1']
        driver_mapping['p2'] = q['_source']['p2']
        driver_mapping['destination'] = q['_source']['destination']

{'altdest1': None,
 'altdest1id': None,
 'altdest2': None,
 'altdest2id': None,
 'ctime': '2016-09-23 18:43:20.743131',
 'destination': None,
 'destinationid': None,
 'id': 6,
 'location': [40.80066864010821, -73.97037331255055],
 'name': 'driver_6',
 'p1': None,
 'p2': None,
 'status': 'idle'}

In [27]:
d_id = 6
    
driver_mapping ={ 
        'name': 'driver_{}'.format(d_id),
        'id': d_id,
        'status': 'idle',
        'location': simulateTrip(d_id, city),
        'ctime': str(datetime.now()),
        'p1': None,
        'p2': None,
        'destination': None,
        'destinationid': None,
        'altdest1': None,
        'altdest1id': None,
        'altdest2': None,
        'altdest2id': None,
    }


In [41]:
q = es.get(index='driver', doc_type='rolling', id=d_id, ignore=[404, 400])
q

{u'_id': u'6',
 u'_index': u'driver',
 u'_source': {u'altdest1': None,
  u'altdest1id': None,
  u'altdest2': None,
  u'altdest2id': None,
  u'ctime': u'2016-09-23T10:32:23.633292Z',
  u'destination': [40.727432567366876, -73.9232224915341],
  u'destinationid': 14,
  u'id': 6,
  u'location': [40.73730956705426, -73.95249180322936],
  u'name': u'driver_6',
  u'p1': None,
  u'p2': None,
  u'status': u'pickup'},
 u'_type': u'rolling',
 u'_version': 2,
 u'found': True}

In [29]:
d.assignPassenger()

True

In [32]:
e

{'altdest1': None,
 'altdest1id': None,
 'altdest2': None,
 'altdest2id': None,
 'ctime': '2016-09-23 18:45:37.283579',
 'destination': [40.727432567366876, -73.9232224915341],
 'destinationid': None,
 'id': 6,
 'location': [40.739778816976106, -73.9232224915341],
 'name': 'driver_6',
 'p1': None,
 'p2': None,
 'status': u'pickup'}

In [39]:
f = getPassenger(14)

In [40]:
f.jsonFormat()

'{"status": "pickup", "altdest2id": "Solomon R. Guggenheim Museum", "altdest2": [40.7831, -73.9592], "altdest1": [40.7114, -74.0125], "altdest1id": "9/11 Memorial & Museum", "destination": [40.758896, -73.98513], "driver": 6, "name": "passenger_14", "destinationid": "Times Square", "location": [40.727432567366876, -73.9232224915341], "id": 14, "match": null, "ctime": "2016-09-23T10:33:35.364037Z"}'

In [37]:
d.jsonFormat()

'{"status": "pickup", "p2": null, "p1": null, "altdest2id": null, "altdest2": null, "altdest1": null, "altdest1id": null, "destination": [40.727432567366876, -73.9232224915341], "name": "driver_6", "destinationid": 14, "location": [40.73730956705426, -73.95249180322936], "id": 6, "ctime": "2016-09-23T10:32:23.633292Z"}'

In [42]:
q

{u'_id': u'6',
 u'_index': u'driver',
 u'_source': {u'altdest1': None,
  u'altdest1id': None,
  u'altdest2': None,
  u'altdest2id': None,
  u'ctime': u'2016-09-23T10:32:23.633292Z',
  u'destination': [40.727432567366876, -73.9232224915341],
  u'destinationid': 14,
  u'id': 6,
  u'location': [40.73730956705426, -73.95249180322936],
  u'name': u'driver_6',
  u'p1': None,
  u'p2': None,
  u'status': u'pickup'},
 u'_type': u'rolling',
 u'_version': 2,
 u'found': True}

In [43]:
d = q['_source']['destination']
c = q['_source']['location']
[c[0] + ((c[0] - d[0])/step_to_dest), d[1] + ((d[1] - d[1])/step_to_dest)]

[40.739778816976106, -73.9232224915341]

In [46]:
q['found'] and (q['_source']['status'] in ['ontrip', 'pickup'])


True

In [47]:
e

{'altdest1': None,
 'altdest1id': None,
 'altdest2': None,
 'altdest2id': None,
 'ctime': '2016-09-23 18:45:37.283579',
 'destination': [40.727432567366876, -73.9232224915341],
 'destinationid': None,
 'id': 6,
 'location': [40.739778816976106, -73.9232224915341],
 'name': 'driver_6',
 'p1': None,
 'p2': None,
 'status': u'pickup'}